In [1]:
# BUILD ITERECT TO USER USING RNNs MODEL , WHEN USER SUBMIT BRAND AND CETEGORY NAME HE GET REPLY PREDICTED PRICE ABOUT 
#THOSE PRODUCT


import pandas as pd
import numpy as np
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# DATASET
df = pd.read_csv(r"C:\Users\nh013\Desktop\FASION DATASET UK AND US\mock_fashion_data_uk_us.csv")

# REMOVE MISING VALUES
df.dropna(inplace=True)

# REMOVE DUPLICATES
df.drop_duplicates(inplace=True)

# REMOVE URLs
df['Customer Reviews'] = df['Customer Reviews'].apply(lambda x: re.sub(r'http\S+', '', x))
df['Product Name'] = df['Product Name'].apply(lambda x: re.sub(r'http\S+', '', x))
df['Social Media Comments'] = df['Social Media Comments'].apply(lambda x: re.sub(r'http\S+', '', x))
df['feedback'] = df['feedback'].apply(lambda x: re.sub(r'http\S+', '', x))
df['Brand'] = df['Brand'].apply(lambda x: re.sub(r'http\S+', '', str(x)))
df['Category'] = df['Category'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

# REMOVE SPECIAL CHERECHTER
df['Customer Reviews'] = df['Customer Reviews'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['Product Name'] = df['Product Name'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['Social Media Comments'] = df['Social Media Comments'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['feedback'] = df['feedback'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['Brand'] = df['Brand'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
df['Category'] = df['Category'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))

# CONVERT TEXT INTO LOWER CASE
df['Customer Reviews'] = df['Customer Reviews'].apply(lambda x: x.lower())
df['Product Name'] = df['Product Name'].apply(lambda x: x.lower())
df['Social Media Comments'] = df['Social Media Comments'].apply(lambda x: x.lower())
df['feedback'] = df['feedback'].apply(lambda x: x.lower())
df['Brand'] = df['Brand'].str.lower()
df['Category'] = df['Category'].str.lower()

# REMOVE STOP WORDS
stop_words = set(stopwords.words('english'))
df['Customer Reviews'] = df['Customer Reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['Product Name'] = df['Product Name'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['Social Media Comments'] = df['Social Media Comments'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['feedback'] = df['feedback'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['Brand'] = df['Brand'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in stop_words]))
df['Category'] = df['Category'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in stop_words]))

# STEMMING
stemmer = PorterStemmer()
df['Customer Reviews'] = df['Customer Reviews'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
df['Product Name'] = df['Product Name'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
df['Social Media Comments'] = df['Social Media Comments'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
df['feedback'] = df['feedback'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
df['Brand'] = df['Brand'].apply(lambda x: ' '.join([stemmer.stem(word) for word in str(x).split()]))
df['Category'] = df['Category'].apply(lambda x: ' '.join([stemmer.stem(word) for word in str(x).split()]))

# FILTER RELEVENT COLUMN
df = df[['Brand', 'Category', 'Price']]

# CREATE INPUT SEQUENCE
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Brand'] + ' ' + df['Category'])
sequences = tokenizer.texts_to_sequences(df['Brand'] + ' ' + df['Category'])
X = pad_sequences(sequences)

# NORMALIZE PRICE COLUMN
scaler = MinMaxScaler()
y = scaler.fit_transform(df['Price'].values.reshape(-1, 1))

# SPLIT DATA 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# RNNs MODEL
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# TRAIN MODEL
model.fit(X_train, y_train, epochs=10, batch_size=32)

# FUNCTION TO PREDICT PRICE
def predict_price(brand, category):
    input_text = brand.lower() + ' ' + category.lower()
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=X.shape[1])
    predicted_price = scaler.inverse_transform(model.predict(input_sequence))
    return predicted_price[0][0]

# INTERECT WITH USER
while True:
    brand = input("Enter the brand: ")
    category = input("Enter the category: ")
    predicted_price = predict_price(brand, category)
    print("Predicted Price:", predicted_price)
    continue_interaction = input("Do you want to continue? (yes/no): ")
    if continue_interaction.lower() != 'yes':
        break


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nh013\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/10
25000/25000 [==============================] - 225s 9ms/step - loss: 0.0839
Epoch 2/10
25000/25000 [==============================] - 226s 9ms/step - loss: 0.0834
Epoch 3/10
25000/25000 [==============================] - 209s 8ms/step - loss: 0.0834
Epoch 4/10
25000/25000 [==============================] - 222s 9ms/step - loss: 0.0834
Epoch 5/10
25000/25000 [==============================] - 231s 9ms/step - loss: 0.0834
Epoch 6/10
25000/25000 [==============================] - 258s 10ms/step - loss: 0.0834
Epoch 7/10
25000/25000 [==============================] - 249s 10ms/step - loss: 0.0834
Epoch 8/10
25000/25000 [==============================] - 243s 10ms/step - loss: 0.0834
Epoch 9/10
25000/25000 [==============================] - 250s 10ms/step - loss: 0.0834
Epoch 10/10
25000/25000 [==============================] - 220s 9ms/step - loss: 0.0834
Enter the brand: Ted Baker
Enter the category: Swimwear
1/1 [==============================] - 1s 681ms/step
Predicted Price: